IMPORTS

In [25]:
import tensorflow as tf
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import os
import pickle
import pandas as pd

INSTACIAR SERVIDOR

In [26]:
app = Flask(__name__)

VERIFICAR SE O MODELO EXISTE

In [27]:
model_path = 'models/CNN_model.keras'
if not os.path.exists(model_path):
  raise FileExistsError(f"O arquivo {model_path} não foi encontrado")

CARREGAR MODELO TREINADO(IMAGE)

In [28]:
try:
  model = load_model(model_path)
  print("Modelo carregado com sucesso")
except Exception as e:
  print(f"Erro ao carregar modelo: {e}")

Modelo carregado com sucesso


CARREGAR MODELO TREINADO(SIMPTOMS)

In [29]:
with open("../api/models/best_model.pkl", "rb") as f:
    best_model = pickle.load(f)

CARREGAR MODELO TREINADO(ENCODERS)

In [30]:
with open("../api/models/encoders.pkl", "rb") as f:
    encoders = pickle.load(f)

INSTANCIAS LABEL_ENCODER

In [31]:
symptoms_encoder = encoders["symptoms"]
sex_encoder = encoders["sex"]
nature_encoder = encoders["nature"]
treatment_encoder = encoders["treatment"]
disease_encoder = encoders["disease"]

PROCESSAR IMAGEM

In [32]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

PREVER DOENÇA

In [33]:
def predict_treatment(novos_sintomas, idade, sexo, natureza, tratamento):
    # Verificar se os novos valores estão nos encoders
    for sintoma in novos_sintomas:
        if sintoma not in symptoms_encoder.classes_:
            raise ValueError(f"Sintoma não reconhecido: {sintoma}")
    if sexo not in sex_encoder.classes_:
        raise ValueError(f"Sexo não reconhecido: {sexo}")
    if natureza not in nature_encoder.classes_:
        raise ValueError(f"Natureza não reconhecida: {natureza}")
    if tratamento not in treatment_encoder.classes_:
        raise ValueError(f"Tratamento não reconhecido: {tratamento}")

    # Codificar os sintomas
    sintomas_codificados = symptoms_encoder.transform(novos_sintomas)

    # Calcular a moda dos sintomas codificados
    sintoma_moda = max(set(sintomas_codificados), key=list(sintomas_codificados).count)

    # Preparar os dados de entrada
    novos_dados = pd.DataFrame(
        {
            "Symptoms": [sintoma_moda],
            "Age": [idade],
            "Sex": [sexo],
            "Nature": [natureza],
            "Treatment": [tratamento],
        }
    )

    # Codificar os dados de entrada
    novos_dados["Sex"] = sex_encoder.transform(novos_dados["Sex"])
    novos_dados["Nature"] = nature_encoder.transform(novos_dados["Nature"])
    novos_dados["Treatment"] = treatment_encoder.transform(novos_dados["Treatment"])

    # Prever o tratamento
    predicao = best_model.predict(novos_dados)
    tratamento_previsto = disease_encoder.inverse_transform(predicao)

    return tratamento_previsto[0]

CRIAR ROTA PREVER IMAGEM

In [34]:
@app.route("/predict_image", methods=["POST"])
def predict():
    if "image" not in request.files:
        return jsonify({"error": "No image file in the request"}), 400

    img_file = request.files["image"]
    img_path = os.path.join("uploads", img_file.filename)
    img_file.save(img_path)

    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)

    response = {
        "predictions": {
            "COVID": f"{prediction[0][0] * 100:.2f}%",
            "Pneumonia": f"{prediction[0][1] * 100:.2f}%",
            "Normal": f"{prediction[0][2] * 100:.2f}%",
        }
    }

    return jsonify(response)

CRIAR ROTA PREVER DOENÇA

In [35]:
@app.route("/predict_symptoms", methods=["POST"])
def predict_symptoms():
    data = request.json
    required_fields = ["symptoms", "age", "sex", "nature", "treatment"]

    for field in required_fields:
        if field not in data:
            return jsonify({"error": f"Missing field: {field}"}), 400

    try:
        sintomas = data["symptoms"]
        if not isinstance(sintomas, list):
            return jsonify({"error": "Field 'symptoms' must be a list"}), 400

        idade = data["age"]
        sexo = data["sex"]
        natureza = data["nature"]
        tratamento = data["treatment"]

        tratamento_previsto = predict_treatment(
            sintomas, idade, sexo, natureza, tratamento
        )
        response = {"predicted_treatment": tratamento_previsto}
        return jsonify(response)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

INICIALIZAR SERVIDOR

In [36]:
if __name__ == "__main__":
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Jun/2024 16:45:46] "POST /predict_symptoms HTTP/1.1" 400 -
127.0.0.1 - - [26/Jun/2024 16:49:09] "POST /predict_symptoms HTTP/1.1" 200 -


{'symptoms': [' coughing'], 'age': 30, 'sex': 'male', 'nature': 'high', 'treatment': 'Antibiotic'}


127.0.0.1 - - [26/Jun/2024 16:50:19] "POST /predict_symptoms HTTP/1.1" 400 -
127.0.0.1 - - [26/Jun/2024 16:51:05] "POST /predict_symptoms HTTP/1.1" 400 -
127.0.0.1 - - [26/Jun/2024 16:52:22] "POST /predict_symptoms HTTP/1.1" 400 -
127.0.0.1 - - [26/Jun/2024 16:56:33] "POST /predict_symptoms HTTP/1.1" 200 -


{'symptoms': [' coughing'], 'age': '12', 'sex': 'male', 'nature': 'low', 'treatment': 'Cough medicine'}


...

127.0.0.1 - - [26/Jun/2024 18:28:02] "POST /predict_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2024 18:29:58] "POST /predict_symptoms HTTP/1.1" 200 -


{'symptoms': [' coughing'], 'age': '12', 'sex': 'male', 'nature': 'high', 'treatment': 'Diuretics'}


127.0.0.1 - - [26/Jun/2024 18:31:02] "POST /predict_symptoms HTTP/1.1" 200 -


{'symptoms': [' coughing'], 'age': '12', 'sex': 'male', 'nature': 'high', 'treatment': 'Omalizumab'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


127.0.0.1 - - [26/Jun/2024 18:31:14] "POST /predict_image HTTP/1.1" 200 -
